In [1]:
import os
import xarray as xr
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import climtas.nci
from dask.distributed import Client, as_completed

In [2]:
def himawari_era5_diff(date):

    date_dt = datetime.strptime(date, "%m-%Y")
    year = date_dt.strftime("%Y")
    month = date_dt.strftime("%m")
    
    ##### Himawari Data
    if date_dt <= datetime.strptime('2019-03-31', '%Y-%m-%d'):
        version = 'v1.0'
    else:
        version = 'v1.1'
    directory=Path(f'/g/data/rv74/satellite-products/arc/der/himawari-ahi/solar/p1s/{version}/{year}/{month}')
    files = sorted(str(p) for p in directory.rglob("*.nc"))
    def _preprocess(ds):
        return ds.drop_vars(set(ds.data_vars) - {'surface_global_irradiance'})
    
    himawari = xr.open_mfdataset(
        files,
        combine='by_coords',
        preprocess=_preprocess,
        engine='h5netcdf',
    )
    himawari = himawari.chunk({'time':100})
    

    ##### ERA5 Data
    era5_dir = [
        Path(f"/g/data/rt52/era5/single-levels/reanalysis/msdwswrf/{year}"),
    ]
    era5_file = [f for d in era5_dir for f in d.glob(f"msdwswrf_era5_oper_sfc_{year}{month}*.nc")][0]
    era5 = xr.open_dataset(era5_file, chunks={'time':100})
    
    # Restrict ERA5 domain to Himawari domain
    era5_aus = era5.sel(
        latitude=slice(-10, -44.5),
        longitude=slice(156.26, 112)
    )
    
    # Get monthly mean
    himawari_clim = himawari.surface_global_irradiance.mean(dim='time')
    era5_clim = era5_aus.msdwswrf.mean(dim='time')
    
    # Regrid ERA5 for comparison
    era5_clim_interp = era5_clim.interp(
        latitude=himawari_clim.latitude,
        longitude=himawari_clim.longitude,
        method='linear'
    )
    
    # Find the difference
    diff = era5_clim_interp - himawari_clim
    
    # Save results
    file_name = f'msdwswrf-era5-himawari_{year}{month}.nc'
    file_path = Path('/g/data/er8/users/cd3022/Irradiance-comparisons/era5-himawari/monthly/')
    file_path.mkdir(parents=True, exist_ok=True)
    diff.to_netcdf(file_path / file_name)
    return f"Finished processing {year}-{month}"

In [3]:
client = Client(
    n_workers=10,
    threads_per_worker=1
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 10
Total threads: 10,Total memory: 95.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40031,Workers: 10
Dashboard: /proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 95.00 GiB
Comm: tcp://127.0.0.1:39329,Total threads: 1
Dashboard: /proxy/34057/status,Memory: 9.50 GiB
Nanny: tcp://127.0.0.1:33939,


In [4]:
if __name__ == '__main__':
    date = '09-2022'
    futures = {}
    future = client.submit(himawari_era5_diff, date)
    futures[future] = f"The job for {date}"
    for future in as_completed(futures):
        _ = future.result()  # Raises if the function failed
    
        # Remove reference to free up memory
        del futures[future]

ERROR 1: PROJ: proj_create_from_database: Open of /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/share/proj failed
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 46.79 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2025-05-27 17:02:22,835 - distributed.worker - WARNING - Compute Failed
Key:       ('chunked_aware_interpnd-7be5a644031cd77dc86210f0781f87d3', 0, 0)
State:     executing
Function:  execute_task
args:      ((subgraph_callable-7d326c09c12f2f51c986210272e3a590, (<function concatenate_axes at 0x1

ValueError: cannot reshape array of size 0 into shape (0,newaxis)